In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from cnn_utils import *
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, GlobalAveragePooling2D
from keras_vggface.vggface import VGGFace


Using TensorFlow backend.


## Verify the installation of Keras VGGFace


In [2]:
import sys
print(sys.executable)


/usr/local/anaconda3/envs/deeplearning/bin/python


In [3]:
# check version of keras_vggface
import keras_vggface
# print version
print(keras_vggface.__version__)

0.6


## Data Preprocessing

In [4]:
df = create_dataframe('/../raw_data/dataset_062920.json')
df.head()

,eyeImage,leftEye,rightEye,y
0,"[[[0.49019607843137253, 0.4627450980392157, 0....","[-0.06916704732662149, 0.19730078279591035, -0...","[0.17425771877389162, 0.21639015714266296, 0.2...","[-0.30000000000000004, -0.3019517795637199]"
1,"[[[0.19607843137254902, 0.1607843137254902, 0....","[-0.17658285534307594, 0.18033866676032262, -0...","[0.06629549190581052, 0.1988126179958256, 0.10...","[0.30000000000000004, -0.9012629161882894]"
2,"[[[0.2196078431372549, 0.19215686274509805, 0....","[-0.17382043840055927, 0.17981051564675177, -0...","[0.06089991895895852, 0.18628686237123016, 0.0...","[0.8999999999999999, -0.9012629161882894]"
3,"[[[0.2196078431372549, 0.17647058823529413, 0....","[-0.15968611544730582, 0.18524077990937093, -0...","[0.08123800929222913, 0.1904861112443328, 0.11...","[-0.30000000000000004, 0.8989667049368542]"
4,"[[[0.5882352941176471, 0.5647058823529412, 0.6...","[-0.07825260742468576, 0.16963032408957468, -0...","[0.1666792245722788, 0.19617187288354976, 0.20...","[-0.30000000000000004, 0.2996555683122848]"


In [5]:
len(df)

846

### Try to use Quaternary encoding to see if we have mislabelled data

In [6]:
# randomly select training examples
x_train, x_validation, y_train, y_validation = create_train_validation(df)

# generate binary y labels
# y_train_binary = create_binary_labels(y_train)
# y_validation_binary = create_binary_labels(y_validation) # generate binary y labels

# generate quaternary y labels
y_train_quaternary = create_quaternary_labels(y_train)
y_validation_quaternary = create_quaternary_labels(y_validation) # generate binary y labels


In [7]:
y_validation_quaternary

,y
6,LR
13,UL
29,LR
33,LR
34,UL
...,...
833,LL
836,UR
838,LL
842,LL


In [8]:
# visualize some data
# plot_eyeImages(x_train,y_train_binary)

In [9]:
# prepare data for feeding in CNN
eyeImage_train, leftEye_train, rightEye_train, quaternary_train = create_tf_data(x_train, y_train_quaternary)
eyeImage_test, leftEye_test, rightEye_test, quaternary_test = create_tf_data(x_validation, y_validation_quaternary)


# encoding the data
# binary_encoder = LabelEncoder()
# binary_encoder.fit(binary_train)
# binary_train = binary_encoder.transform(binary_train)
# binary_test = binary_encoder.transform(binary_test)

# # one-hot encoding the data
# binary_train = to_categorical(binary_train)
# binary_test = to_categorical(binary_test)


In [10]:
quaternary_train

array(['UL', 'UR', 'UR', 'LL', 'LL', 'LL', 'UR', 'LL', 'LR', 'LR', 'LR',
       'UR', 'UL', 'LL', 'UL', 'LL', 'UR', 'UR', 'LL', 'LL', 'LR', 'UL',
       'UL', 'UL', 'UR', 'LR', 'UR', 'LR', 'UR', 'LR', 'LR', 'LL', 'UR',
       'UL', 'UR', 'LL', 'UL', 'UR', 'LR', 'UL', 'LL', 'UR', 'UL', 'LR',
       'UL', 'LL', 'UR', 'LR', 'LL', 'UR', 'LR', 'LL', 'LL', 'LR', 'UL',
       'UR', 'LR', 'UR', 'UL', 'UL', 'LL', 'UR', 'LR', 'LR', 'UL', 'LR',
       'UL', 'LL', 'LR', 'LL', 'UR', 'UR', 'LR', 'UL', 'LL', 'UR', 'LL',
       'LR', 'LL', 'LL', 'LL', 'UL', 'LL', 'UL', 'UL', 'UR', 'LR', 'LL',
       'LR', 'UL', 'LR', 'UR', 'LR', 'UR', 'LL', 'LR', 'LR', 'UL', 'LL',
       'LR', 'UR', 'LL', 'UL', 'UL', 'UR', 'LL', 'LL', 'UR', 'UL', 'UL',
       'UR', 'LR', 'LL', 'LL', 'LL', 'LL', 'UL', 'UR', 'LR', 'LL', 'LR',
       'UR', 'LL', 'UL', 'UL', 'UR', 'UR', 'LR', 'LL', 'UR', 'LL', 'LR',
       'UL', 'LL', 'UR', 'UL', 'LL', 'UL', 'UL', 'LR', 'LL', 'UR', 'UR',
       'UL', 'UL', 'UR', 'LL', 'UL', 'LR', 'LL', 'L

In [11]:
quaternary_encoder = LabelEncoder()
quaternary_encoder.fit(quaternary_train)
quaternary_train = quaternary_encoder.transform(quaternary_train)
quaternary_test = quaternary_encoder.transform(quaternary_test)

# one-hot encoding the data
quaternary_train = tf.keras.utils.to_categorical(quaternary_train)
quaternary_test = tf.keras.utils.to_categorical(quaternary_test)


In [12]:
quaternary_train

array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       ...,
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.]], dtype=float32)

## VGGFace2 Transfer Learning

In [13]:
baseModel = VGGFace(model='resnet50',input_shape=(224,224,3),include_top=False,weights='vggface', pooling=None)

In [14]:
baseModel.summary()

                 
__________________________________________________________________________________________________
conv4_1_1x1_reduce (Conv2D)     (None, 14, 14, 256)  131072      activation_22[0][0]              
__________________________________________________________________________________________________
conv4_1_1x1_reduce/bn (BatchNor (None, 14, 14, 256)  1024        conv4_1_1x1_reduce[0][0]         
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 14, 14, 256)  0           conv4_1_1x1_reduce/bn[0][0]      
__________________________________________________________________________________________________
conv4_1_3x3 (Conv2D)            (None, 14, 14, 256)  589824      activation_23[0][0]              
__________________________________________________________________________________________________
conv4_1_3x3/bn (BatchNormalizat (None, 14, 14, 256)  1024        conv4_1_3x3[0][0]         

In [15]:
headModel = baseModel.get_layer('avg_pool').output
headModel = Flatten(name='flatten')(headModel)
headModel = Dense(4, activation='softmax', name='classifier')(headModel)


In [16]:
model = Model(inputs=baseModel.input, outputs=headModel)
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])



In [17]:
model.summary()

ce[0][0]         
__________________________________________________________________________________________________
activation_23 (Activation)      (None, 14, 14, 256)  0           conv4_1_1x1_reduce/bn[0][0]      
__________________________________________________________________________________________________
conv4_1_3x3 (Conv2D)            (None, 14, 14, 256)  589824      activation_23[0][0]              
__________________________________________________________________________________________________
conv4_1_3x3/bn (BatchNormalizat (None, 14, 14, 256)  1024        conv4_1_3x3[0][0]                
__________________________________________________________________________________________________
activation_24 (Activation)      (None, 14, 14, 256)  0           conv4_1_3x3/bn[0][0]             
__________________________________________________________________________________________________
conv4_1_1x1_increase (Conv2D)   (None, 14, 14, 1024) 262144      activation_24[0][0]       

In [18]:
history_1= model.fit(
	x = eyeImage_train,
	y = quaternary_train,
    validation_split=0.2,
    batch_size=32,
	epochs=20)


Train on 541 samples, validate on 136 samples
Epoch 1/20
 32/541 [>.............................] - ETA: 17:29 - loss: 0.7665 - accuracy: 0.2812

In [ ]:
# summarize history for accuracy
plt.plot(history_1.history['accuracy'])
plt.plot(history_1.history['val_accuracy'])
plt.title('model accuracy of vggface2 1')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')
fig1 = plt.gcf()
plt.show()
plt.draw()
fig1.savefig('results/vggface2_1_acc.png')

# summarize history for loss
plt.plot(history_1.history['loss'])
plt.plot(history_1.history['val_loss'])
plt.title('model loss of vggface2 1tf.keras.utils.to_categorical
')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')
fig1 = plt.gcf()
plt.show()
plt.draw()
fig1.savefig('results/vggface2_1_loss.png')


In [ ]:
history_2 = model.evaluate(eyeImage_test, quaternary_test)

## Updated version of `create_tf_data`, July 1, 2020
Substract every image from the average image, every landmarks from the average landmarks positions

### Get the mean image of the training dataset

In [ ]:
mean_image = np.mean(eyeImage_train, axis = 0)
# meadian_image = np.median(eyeImage_train, axis = 0)

In [ ]:
plt.imshow(mean_image)
# plt.savefig("results/average_img.png")

### Updated version of `create_tf_data`

In [ ]:
def create_tf_data(X, Y):
    """
    take in the X and Y and transform each column into np array
    Every column in X will be substracted from its mean
    """
    
    eyeImage = np.stack(X['eyeImage'].to_numpy())
    image_mean = np.mean(eyeImage, axis=0)
    eyeImage = eyeImage-image_mean

    leftEye = np.stack(X['leftEye'].to_numpy())
    leftEye_mean = np.mean(leftEye, axis=0)
    leftEye = leftEye-leftEye_mean

    rightEye = np.stack(X['rightEye'].to_numpy())
    rightEye_mean = np.mean(rightEye, axis=0)
    rightEye = rightEye-rightEye_mean

    y = np.stack(Y['y'].to_numpy())
    return eyeImage, leftEye, rightEye, y

### Verify that we substract the mean correctly

In [ ]:
_,left,_,_ = create_tf_data(x_train, y_train_binary)

In [ ]:
np.sum(left,axis=0)